<a href="https://colab.research.google.com/github/Abdul-AI-creator/Abdul-AI-creator/blob/main/Pytorch_intermediat(deep_residual_network).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

**device configuration**

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

**HyperParameters**

In [4]:
n_epochs =80
batch_size =100
learning_rate = 0.003

**Image Preprocessing Modules**

In [5]:
transform =transforms.Compose(
    [transforms.Pad(4),
     transforms.RandomHorizontalFlip(),
     transforms.RandomCrop(32),
     transforms.ToTensor()]
)
transform

Compose(
    Pad(padding=4, fill=0, padding_mode=constant)
    RandomHorizontalFlip(p=0.5)
    RandomCrop(size=(32, 32), padding=None)
    ToTensor()
)

**CIFAR10 DATASET**

In [6]:
train_ds = torchvision.datasets.CIFAR10(root = 'data' ,transform=transform ,download=True,train=True)
test_ds = torchvision.datasets.CIFAR10(root='data' ,train=False ,transform=transforms.ToTensor())
train_ds

100%|██████████| 170M/170M [00:13<00:00, 12.4MB/s]


Extracting data/cifar-10-python.tar.gz to data


Dataset CIFAR10
    Number of datapoints: 50000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               Pad(padding=4, fill=0, padding_mode=constant)
               RandomHorizontalFlip(p=0.5)
               RandomCrop(size=(32, 32), padding=None)
               ToTensor()
           )

**Data Loader**

In [7]:
train_loader = torch.utils.data.DataLoader(dataset=train_ds ,batch_size=batch_size ,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_ds ,batch_size=batch_size ,shuffle=False)

**3x3 Convolution**

In [8]:
def conv3x3(in_channels ,out_channels ,stride=1):  ##Stride = 1 (filter slides one pixel at a time).
   return nn.Conv2d(in_channels ,out_channels ,kernel_size=3 ,stride=stride ,padding=1 ,bias=False) #Each side of the
                                                                                                 #input (top, bottom, left, right) gets 1 row or column of zeros added.

***Residual Block***

In [9]:
class ResidualBlock(nn.Module):
  def __init__(self ,in_channels ,out_channels ,stride=1 ,downsample=None):
    super(ResidualBlock ,self).__init__()
    self.conv1=conv3x3(in_channels ,out_channels ,stride)
    self.bn1 =nn.BatchNorm2d(out_channels)
    self.relu = nn.ReLU(inplace=True)
    self.conv2 =conv3x3(out_channels ,out_channels)
    self.bn2 =nn.BatchNorm2d(out_channels)
    self.downsample =downsample

  def forward(self,x):
    residual =x
    out =self.conv1(x)
    out = self.bn1(out)
    out =self.relu(out)
    out =self.conv2(out)
    out=self.bn2(out)
    if self.downsample:
      residual = self.downsample(x)
    out += residual
    out =self.relu(out)
    return out

**ResNet**

In [10]:
class ResNet(nn.Module):
  def __init__(self ,block ,layers ,n_classes=10):
    super(ResNet ,self).__init__()
    self.in_channels =16
    self.conv = conv3x3(3, 16)
    self.bn = nn.BatchNorm2d(16)
    self.relu = nn.ReLU(inplace=True)
    self.layer1 = self.make_layer(block ,16 ,layers[0])
    self.layer2 = self.make_layer(block ,32 ,layers[1] ,2)
    self.layer3 = self.make_layer(block ,64 ,layers[2] ,2)
    self.avg_pool = nn.AvgPool2d(8)
    self.fc = nn.Linear(64 ,n_classes)

  def make_layer(self, block ,out_channels ,blocks ,stride=1):
    downsample=None
    if (stride != 1) or (self.in_channels != out_channels):
      downsample =nn.Sequential(conv3x3(self.in_channels ,out_channels ,stride=stride), nn.BatchNorm2d(out_channels))

    layers =[]
    layers.append(block(self.in_channels ,out_channels ,stride ,downsample))
    self.in_channels =out_channels
    for i in range(1 ,blocks):
      layers.append(block(out_channels ,out_channels))
    return nn.Sequential(*layers)

  def forward(self,x):
    out =self.conv(x)
    out =self.bn(out)
    out =self.relu(out)
    out =self.layer1(out)
    out =self.layer2(out)
    out =self.layer3(out)
    out= self.avg_pool(out)
    out= out.view(out.size(0), -1)
    out =self.fc(out)
    return out
model =ResNet(ResidualBlock, [2,2,2]).to(device)
model

ResNet(
  (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=

**Loss and Optimizer**

In [11]:
criterion =nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters() ,lr =learning_rate)

**For updating learning rate**

In [12]:
def update_lr(optimizer ,lr):
  for param_group in optimizer.param_groups:
    param_group['lr'] = lr


**Train the Model**

In [ ]:
total_step =len(train_loader)
curr_lr =learning_rate
for epoch in range(n_epochs):
  for i ,(images ,labels) in enumerate(train_loader):
     images =images.to(device)
     labels =labels.to(device)
     # forward pass
     outputs =model(images)
     loss =criterion(outputs ,labels)

     # Backward and optimize
     optimizer.zero_grad()
     loss.backward()
     optimizer.step()
     if (i+1) % 100 == 0:
      print("Epoch [{}/{}], Step [{}/{}] ,Loss: {:.4f}".format(epoch+1 ,n_epochs, i+1 ,total_step ,loss.item()))

  if (epoch +1) % 20 ==0:
    curr_lr /= 3
    update_lr(optimizer ,curr_lr)

Epoch [1/80], Step [100/500] ,Loss: 1.7003
Epoch [1/80], Step [200/500] ,Loss: 1.5223
Epoch [1/80], Step [300/500] ,Loss: 1.3193
Epoch [1/80], Step [400/500] ,Loss: 1.1921
Epoch [1/80], Step [500/500] ,Loss: 1.2635
Epoch [2/80], Step [100/500] ,Loss: 1.1594
Epoch [2/80], Step [200/500] ,Loss: 1.0143
Epoch [2/80], Step [300/500] ,Loss: 0.9772
Epoch [2/80], Step [400/500] ,Loss: 0.9970
Epoch [2/80], Step [500/500] ,Loss: 0.8887
Epoch [3/80], Step [100/500] ,Loss: 0.9387
Epoch [3/80], Step [200/500] ,Loss: 0.9213
Epoch [3/80], Step [300/500] ,Loss: 1.0041
Epoch [3/80], Step [400/500] ,Loss: 0.9138
Epoch [3/80], Step [500/500] ,Loss: 0.8463
Epoch [4/80], Step [100/500] ,Loss: 0.7033
Epoch [4/80], Step [200/500] ,Loss: 0.9506
Epoch [4/80], Step [300/500] ,Loss: 0.7429
Epoch [4/80], Step [400/500] ,Loss: 0.7639
Epoch [4/80], Step [500/500] ,Loss: 0.7459
Epoch [5/80], Step [100/500] ,Loss: 0.5443
Epoch [5/80], Step [200/500] ,Loss: 0.6410
Epoch [5/80], Step [300/500] ,Loss: 0.9295
Epoch [5/80

**Test model**

In [ ]:
model.eval()
with torch.no_grad():
  correct=0
  total=0
  for images ,labels in test_loader:
    images =images.to(device)
    labels = labels.to(device)
    outputs =model(images)
    _, predicted = torch.max(outputs.data ,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print('Accuracy of model on Test Images: {} %'.format(100 * correct / total))

torch.save(mode.state_dict() ,'deep_residual_net')